In [83]:
# Import packages
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
import statsmodels.api as sm
import math as ma

In [84]:
def indent():
    print()
    print()

In [85]:
# Load Fama-French factor data
ff3f = pdr.DataReader('F-F_Research_Data_Factors', 'famafrench', '1926-01-01', '2024-03-31')[0]/100
ff3f.head(5)
ff3f.iloc[:,:]

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07,0.0296,-0.0256,-0.0243,0.0022
1926-08,0.0264,-0.0117,0.0382,0.0025
1926-09,0.0036,-0.0140,0.0013,0.0023
1926-10,-0.0324,-0.0009,0.0070,0.0032
1926-11,0.0253,-0.0010,-0.0051,0.0031
...,...,...,...,...
2023-09,-0.0524,-0.0251,0.0152,0.0043
2023-10,-0.0319,-0.0387,0.0019,0.0047
2023-11,0.0884,-0.0002,0.0164,0.0044


In [86]:
    # Download monthly prices (keep only Adjusted Close prices)
ticker = 'NVDA'   
    
firm_prices = yf.download(ticker, '2015-01-01', '2020-12-31', interval = '1mo')['Adj Close'].dropna().to_frame()

    # Calculate monthly returns, drop missing, convert from Series to DataFrame
firm_ret = firm_prices.pct_change().dropna()

    # Rename "Adj Close" to "TSLA"
firm_ret.rename(columns = {'Adj Close': 'Returns'}, inplace = True)

    # Convert index to monthly period date
firm_ret.index = firm_ret.index.to_period('M')
firm_ret.head(2)

[*********************100%%**********************]  1 of 1 completed


,Returns
Date,
2015-02,0.148958
2015-03,-0.047571


In [87]:
# Merge the two datasets
data = firm_ret.join(ff3f)
data['const'] = 1
data.head(2)

,Returns,Mkt-RF,SMB,HML,RF,const
Date,,,,,,
2015-02,0.148958,0.0613,0.0063,-0.0186,0.0,1
2015-03,-0.047571,-0.0112,0.0304,-0.0037,0.0,1


In [88]:
# Set up the data
    # Dependent variable (left side of the equal sign)
y = data['Returns'] - data['RF']
y.head(2)

Date
2015-02    0.148958
2015-03   -0.047571
Freq: M, dtype: float64

In [89]:
    # Independent variable(s) (right side of the equal sign)
X = data[['const','Mkt-RF']]
X.head(2)

,const,Mkt-RF
Date,,
2015-02,1,0.0613
2015-03,1,-0.0112


In [90]:
# Run single factor regression and store results in "res" object
res = sm.OLS(y,X).fit()
print(res.summary())
print("Reminder that 1 - Rsquared equals the percentage of the firms total risk that can be diversified away (idiosyncratic)")

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     21.31
Date:                Sun, 24 Mar 2024   Prob (F-statistic):           1.75e-05
Time:                        22:54:08   Log-Likelihood:                 58.819
No. Observations:                  71   AIC:                            -113.6
Df Residuals:                      69   BIC:                            -109.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0397      0.013      3.026      0.0

In [91]:
#Adjusted Betas
l = 0.33
hBeta = res.params[1]
Adj_Beta = hBeta * (1-l) + l
print(f"The Adjusted Beta For {ticker} is: {Adj_Beta}")

The Adjusted Beta For NVDA is: 1.1972327976486767


In [92]:
tot_risk  = y.var()

print(f"The total risk of {ticker} equals: {tot_risk}")

indent()

beta = res.params['Mkt-RF']
sys_risk  = (beta**2) * data['Mkt-RF'].var() 

print(f"The systematic risk equals: {sys_risk}")

#Idiosyncratic Risk 
idio_risk = tot_risk - sys_risk
print(f"The idiosyncratic risk equals: {idio_risk}")

#As percentages
# Print as percentages of total risk
pct_sys_risk = sys_risk / tot_risk
pct_idio_risk = idio_risk / tot_risk
print(f'\n percent systematic risk = {pct_sys_risk: .4f} \n percent idiosyncratic risk = {pct_idio_risk: .4f}')

The total risk of NVDA equals: 0.014825033380208923


The systematic risk equals: 0.0034980933292737292
The idiosyncratic risk equals: 0.011326940050935194

 percent systematic risk =  0.2360 
 percent idiosyncratic risk =  0.7640


In [93]:
#Single Factor Cost of Equity
Emrp = sum(ff3f['Mkt-RF']) / len(ff3f['Mkt-RF'])
Rf = 0.0047 #Set this depending if RF is given to you in question, or if the question asks to use to most recent number
COE = (Rf + hBeta * Emrp) *12 * 100
COE_r = round(COE, 3)
print(f"The Annualized Cost of Equity for {ticker} is: {COE_r}%")

The Annualized Cost of Equity for NVDA is: 16.172%


In [94]:
#Single Factor P Val Interpretation
p_vals1 = res.pvalues
alpha = res.params[0]
if p_vals1['const'] < 0.01: #Set your significance level here
    print(f"Reject the null that true alpha is 0: we should reject the null hypothesis that the true alpha is 0")
    if alpha > 0:
        print(f"{ticker} is undervalued")
    else:
        print(f"{ticker} is overvalued")
else:
    print(f"Cannot reject the null that alpha is 0: we do not have sifficient eidence that the stock is mispriced")

Reject the null that true alpha is 0: we should reject the null hypothesis that the true alpha is 0
NVDA is undervalued


In [95]:
#Estimate the Fama-French three factor model using the data gathered above


# Set up X variables
X3 = data[['const','Mkt-RF','SMB','HML']]
# Run regression
res3 = sm.OLS(y,X3).fit()
print("Fama French 3 Factor Regression")
print(res3.summary())

Fama French 3 Factor Regression
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.220
Method:                 Least Squares   F-statistic:                     7.585
Date:                Sun, 24 Mar 2024   Prob (F-statistic):           0.000193
Time:                        22:54:08   Log-Likelihood:                 59.645
No. Observations:                  71   AIC:                            -111.3
Df Residuals:                      67   BIC:                            -102.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.035

In [96]:
#Three Factor Adjusted Beta
h3Beta = res3.params[1]
Adj_Beta = h3Beta * (1-l) + l
print(f"The Adjusted Three Factor Beta For {ticker} is: {Adj_Beta}")

The Adjusted Three Factor Beta For NVDA is: 1.2432062061504128


In [97]:
# Total risk of our stock (variance) in 3 factor (also works in single factor)

print(f"Again, the total risk of {ticker} equals: {tot_risk}")

indent()

# Systematic Risk
beta3 = res3.params['Mkt-RF']
sys_risk3  = (beta3**2) * data['Mkt-RF'].var() 

print(f"The 3 factor systematic risk equals: {sys_risk3}")

#Idiosyncratic Risk 
idio_risk3 = tot_risk - sys_risk
print(f"The 3 factor idiosyncratic risk equals: {idio_risk3}")

Again, the total risk of NVDA equals: 0.014825033380208923


The 3 factor systematic risk equals: 0.003878802911547808
The 3 factor idiosyncratic risk equals: 0.011326940050935194


In [98]:
# Print as percentages of total risk
pct_sys_risk = sys_risk3 / tot_risk
pct_idio_risk = idio_risk3 / tot_risk
print(f'\n 3 factor percent systematic risk = {pct_sys_risk: .4f} \n 3 factor percent idiosyncratic risk = {pct_idio_risk: .4f}')


 3 factor percent systematic risk =  0.2616 
 3 factor percent idiosyncratic risk =  0.7640


In [99]:
#Pvalue and Alpha for 3 factor model
p_vals = res3.pvalues
if p_vals['const'] < 0.01: #Set your significance level here
    print(f"Reject the null that true alpha is 0: we should reject the null hypothesis that the true alpha is 0")
else:
    print(f"Cannot reject the null that alpha is 0: we do not have sifficient eidence that the stock is mispriced")

Cannot reject the null that alpha is 0: we do not have sifficient eidence that the stock is mispriced


In [100]:
#Cost of Equity in 3 factor
#We already have our risk free rate variabe "Rf" 
#and the expected Market RIsk Premium "Ermp" and beta for the stock "hBeta"
#We now need the exp ret for our other two factors and their betas
#also we need to update the beta "hBeta" to "h3Beta"

#Expected Returns
ErSMB = sum(ff3f['SMB']) / len(ff3f['SMB'])
ErHML = sum(ff3f['HML']) / len(ff3f['HML'])

#SMB, HML, and 3h Betas
Bhml = res3.params[3]
Bsmb = res3.params[2]
h3Beta = res3.params[1]
print(f" HML - {Bhml} and SMB - {Bsmb} and the market beta {h3Beta}")


f3COE = Rf + (h3Beta * Emrp) + (Bsmb * ErSMB) + (Bhml * ErHML)
Af3COE = (f3COE * 12) * 100


print(f"The **Annualized*** Cost of Equity for {ticker} is: {Af3COE.round(3)}%")


 HML - -0.5274408572132163 and SMB - -0.002744685826872817 and the market beta 1.3629943375379296
The **Annualized*** Cost of Equity for NVDA is: 14.537%
